In [1]:
import hashlib
import json
import os
import re
import ssl
from datetime import datetime
from datetime import date
from datetime import timedelta
from glob import glob
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import unicodedata
from bs4 import BeautifulSoup
from nltk import word_tokenize
from nltk.stem.snowball import FrenchStemmer

import requests



def get_access_token():
    """
    Obtient les tokens d'accès à l'api Legifrance
    :return: les tokens
    """
    token_url = "https://sandbox-oauth.piste.gouv.fr/api/oauth/token"
    # client (application_prod) credentials
    client_id = "ID"
    client_secret = "MDP"
    # step A, B - single call with client credentials as the basic auth header - will return access_token
    data = {'grant_type': 'client_credentials',
            'Content-Type': 'application/x-www-form-urlencoded',
            'Accept': 'application/json',
            'scope': 'openid'
            }
    ssl._create_default_https_context = ssl._create_unverified_context
    access_token_response = requests.post(token_url, data=data, auth=(client_id, client_secret),verify=False)
    token_response = access_token_response.status_code
    tokens = json.loads(access_token_response.text)
    return tokens

In [2]:
def get_api_dict(url, tokens, code_ancien_id):
    """
    Obtient la réponse de l'api légifrance pour une url sous forme d'un json
    :param url: l'url traitée
    :param tokens: tokens d'authentification à l'api
    :param logger: un logger
    :param code_ancien_id: le code ancien id si nécessaire à l'appel
    :return: un json contenant la réponse de l'api
    """
    url = url.strip()
    url = url + '/' if url[-1] != '/' else url
    id_doc = url.split("/")[-2]
    methode_type = url.split("/")[3]
    alternative={}
    date_rq = datetime.today().strftime('%Y-%m-%d')
    api_call_headers = {'Authorization': 'Bearer ' + tokens['access_token'],
                        "Content-Type": "application/json", "accept": "application/json"}
    if methode_type == 'loda':
        test_api_url = 'https://sandbox-api.piste.gouv.fr/dila/legifrance/lf-engine-app/consult/lawDecree'
        payload = "{\"date\": \"%s\",\"textId\": \"%s\"}" % (date_rq, id_doc)

    elif methode_type == 'jorf':
        test_api_url = "https://sandbox-api.piste.gouv.fr/dila/legifrance/lf-engine-app/consult/jorf" #'https://api.piste.gouv.fr/dila/legifrance/lf-engine-app/consult/jorf'
        payload = "{\"textCid\": \"%s\"}" % id_doc
        alternative = { "textCid": id_doc }

    elif methode_type == 'codes':
        if code_ancien_id != "":
            test_api_url = 'https://sandbox-api.piste.gouv.fr/dila/legifrance/lf-engine-app/consult/getCodeWithAncienId'
            payload = "{\"ancienId\": \"%s\"}" % code_ancien_id
        else:
            test_api_url = 'https://sandbox-api.piste.gouv.fr/dila/legifrance/lf-engine-app/consult/code'
            textId = url.split("/")[-3]
            payload = "{\"date\": \"%s\",\"sctCid\": \"%s\",\"textId\": \"%s\"}" % (date_rq, id_doc, textId)
    else : 
        return
    api_call_response = requests.post(test_api_url, data=payload, headers=api_call_headers, verify=False)
    return api_call_response.json()
def get_date_parution(date_parution):
    """
    Obtient la valeur JJ-MM-AAAA à partir d'un long
    :param date_parution: version format long d'une date
    :return: la date sous le format JJ-MM-AAAA
    """
    timestamp = date_parution / 1000
    if timestamp >= 0:
        date_doc = datetime.fromtimestamp(timestamp)
    else:
        date_doc = datetime(1970, 1, 1) + timedelta(seconds=int(timestamp))
    date_doc = str(date_doc.date())
    if date_doc == "2999-01-01":
        return "1900-01-01"
    else:
        return date_doc

def traitement_section(ff):
    """
    Traitement d'une section d'un json pouvant contenir des sous section et/ou
    des articles
    si sous-sections: on applique cette fonction à chacune des sous sections
    si articles: applique tratement_article à chaque article
    :param ff: le json sous forme de dictionnaire
    :param dict_json: le dictionnaire à compléter
    """

    for elem in ff['sections']:
        traitement_section(elem)

    for art in ff['articles']:
        traitement_article(art)


def traitement_article(article):
    """
    Traitement d'un article
    :param article; le dict de l'article
    :param dict_json: le dictionnaire à compléter
    """

    etat = article['etat']
    if etat != "ABROGE":
        num = article['num']
        content = article['content']
        print(num)
        print(content)





In [5]:
lien = "https://www.legifrance.gouv.fr/jorf/id/JORFTEXT000049171259"
api_result = get_api_dict(lien, get_access_token(), "")

In [6]:
traitement_section(api_result)

1
<p><br/>Délégation est donnée à l'effet de signer au nom du ministre de la transformation et de la fonction publiques ainsi qu'au nom du Premier ministre, tous actes relevant des attributions de leurs département, et notamment les conventions, commandes et certifications de services faits d'un montant strictement inférieur à 208 000 € hors taxes, aux agents suivants :<br/>M. Ishan Bhojwani, agent contractuel, chef du département « Incubateur de services numériques » (ISN) ;<br/>Mme Marielle Chrisment, lieutenante-colonelle de gendarmerie, cheffe du département « Etalab » ;<br/>M. Guy Duplaquet, ingénieur général des mines, chef du département « Infrastructures et services opérés » (ISO) ;<br/>M. Florian Delezenne, agent contractuel, chef du département « Opérateur de produits interministériels » (OPI) ;<br/>Mme Cornelia Findeisen, agente contractuelle, cheffe du département « RH de la filière numérique de l'Etat » (DRHN).</p>
2
<p><br/>Délégation est donnée à l'effet de signer au nom